In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt

def covariance(i, H):
    if i==0:
        return 1
    else:
        return ((i-1)**(2*H) - 2*i**(2*H) + (i+1)**(2*H))/2

def fBM_Levinson(m, H, L=1, cumm=1,_seed=None):
    k=np.array(range(m))
    if _seed:
        np.random.seed(_seed)
    scaling = (L/m) ** (2*H)
    
    # -- Covariance
    cov = [covariance(i, H) for i in range(m)]
    
    # -- Initialization of the algorithm
    y = np.random.normal(0,1,m)
    fGn = np.zeros(m)
    v1 = np.array(cov)
    v2 = np.array([0] + cov[1:] + [0])
    k = v2[1]
    aa = np.sqrt(cov[0])
    
    # -- Levinson's algorithm
    for j in range(1,m):
        aa = aa * np.sqrt(1 - k*k)
        v = k * v2[j:m] + v1[j-1:m-1]
        v2[j:m] = v2[j:m] + k * v1[j-1:m-1]
        v1[j:m] = v
        bb = y[j] / aa
        fGn[j:m] = fGn[j:m] + bb * v1[j:m]
        k = -v2[j+1]/(aa*aa)
    
    # -- scaling and output
    for i in range(m):
        fGn[i] = scaling * fGn[i]
        if cumm and i>0:
            fGn[i] = fGn[i] + fGn[i-1]
    
    return fGn


In [3]:
ret = fBM_Levinson(m=1000, H=0.6, cumm=0, L=10)
sample = np.cumprod(1+ret)
plt.plot(sample)

In [4]:
# H-P Filtering

# Full Sample
def apply_hp_filter(sample):
    x = np.zeros((len(sample)-2, len(sample)), dtype=np.int)
    for i, v in enumerate((1,-2, 1)):
        np.fill_diagonal(x[:,i:], v)

    I = np.eye(len(sample))
    states = np.dot(np.linalg.inv(I + 10*np.dot(x.T, x)), sample)
    return states

# Halfsample
s = apply_hp_filter(sample)
s_2 = apply_hp_filter(sample[:int(len(sample)/2)])

plt.plot(sample, color='gray', lw=3, alpha=0.4)
plt.plot(s)
plt.plot(s_2)

In [21]:
# Kernel Ridge regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge


X = np.arange(len(sample))[:, None]
poly = PolynomialFeatures(5)
Feat = poly.fit_transform(X)

clf = KernelRidge(alpha=1.0)
clf.fit(Feat,sample) 
y_pred = clf.predict(Feat)

plt.plot(sample)
plt.plot(y_pred)

C:\Users\tamaghna\Anaconda4\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
